In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import interpolate

In [ ]:
from sqlalchemy import create_engine  # connecting to local postgres database

In [ ]:
engine = create_engine("postgresql://postgres@db/postgres")

raw_data = pd.read_sql_query(
    """
    SELECT 
        *, 
        date_trunc('day', datetime) as date,
        AVG("close") OVER(ORDER BY symbol, datetime ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS close5
    FROM stocks_5m_rth 
    WHERE datetime > '2019-05-01' AND datetime < '2019-06-01'
    AND symbol IN ('TSLA', 'AMZN', 'CAR', 'GOOGL', 'AAPL', 'MSFT', 'BAC', 'BA', 'C', 'DB')
    ORDER BY datetime
    """,
    con=engine,
)

trading_days = raw_data.date.unique()
trading_symbols = raw_data.symbol.unique()


# AND symbol IN ('TSLA', 'AMZN', 'CAR', 'GOOGL', 'AAPL', 'MSFT', 'BAC', 'BA', 'C', 'DB')
# raw_data
# print(trading_days)
print(trading_symbols)

In [ ]:
print(trading_days)

In [ ]:
def getmydata(myday, mysymbol):
    ts_data = raw_data[(raw_data["date"] == myday) & (raw_data["symbol"] == mysymbol)]
    ts_data = ts_data.reset_index(
        drop=True
    )  # This changes the index to a contiguous range starting from 0
    return ts_data


# test this function --> success
# getmydata("2020-01-21", "AAPL")
# ts_price = getmydata("2020-01-21", "AAPL")
myday = "2019-05-20"
mysymbol = "AAPL"
ts_price = getmydata(myday, mysymbol)
ts_price

In [ ]:
# old definition
def fit_function(timeframe):
    if timeframe < 20:
        return False
    now = timeframe + 1
    x = np.linspace(0, now + 19, num=now + 20, endpoint=True)
    x_realtime = x[now - 20 : now]
    x_realtime_odd = x_realtime[1::2]
    x_realtime_even = x_realtime[::2]
    y = ts_price["close"]
    y_realtime = y[now - 20 : now]
    y_realtime_odd = y_realtime[1::2]
    y_realtime_even = y_realtime[::2]
    # print(x_realtime_odd)
    ff = interpolate.InterpolatedUnivariateSpline(x_realtime_odd, y_realtime_odd, k=5)
    return ff

In [ ]:
timeframe = 25
# Think about now = timeframe + 1 if you want the opening price
now = timeframe
x = np.linspace(0, now + 19, num=now + 20, endpoint=True)
x_realtime = x[now - 20 : now]
x_realtime_odd = x_realtime[1::2]
x_realtime_even = x_realtime[::2]
y = ts_price["close"]
# ysmooth10 = ts_price["close10"]
ysmooth5 = ts_price["close5"]
y_realtime = y[now - 20 : now]
y_realtime_odd = y_realtime[1::2]
y_realtime_even = y_realtime[::2]
# print(x_realtime_odd)
# ff = interpolate.InterpolatedUnivariateSpline(x_realtime_even, y_realtime_even, k=4)
# ff_derivative = ff.derivative()
ff = interpolate.InterpolatedUnivariateSpline(x_realtime, ysmooth5[now - 20 : now], k=4)

xnew = np.linspace(now - 21, now, num=500, endpoint=True)
ynew = ff(xnew)
yderivative = ff_derivative(xnew)
thisfigure = plt.figure()
plt.plot(
    x[now - 21 : now + 4],
    y[now - 21 : now + 4],
    "b",
    xnew,
    ynew,
    "r",
    x[now - 21 : now + 4],
    ysmooth10[now - 21 : now + 4],
    "y",
    x[now - 21 : now + 4],
    ysmooth5[now - 21 : now + 4],
    "g",
)
plt.axvline(now, linewidth=2, color="black", linestyle="--")
plt.legend(["Points", "Spline"])
plt.title("InterpolatedUnivariateSpline")
plt.show()

In [385]:
from scipy import optimize
from scipy.misc import derivative


def graph(myday, mysymbol, timeframe):

    now = timeframe
    x = np.linspace(0, now + 19, num=now + 20, endpoint=True)
    y = ts_price["close"]
    # ysmooth10 = ts_price["close10"]
    ysmooth5 = ts_price["close5"]

    def test_func_affine(x, m, k):
        return m * x + k

    def test_func_sin(x, a, b, c):
        return a * np.sin(b * x + c)

    def test_func_wave(x, a, b, c, d):
        return a * np.sin(b * x + c) + d

    # def test_func(x, a, b, c, d, m, k):
    #   return m * x + k + a * np.sin(b * x + c) + d

    affineparams, affineparams_covariance = optimize.curve_fit(
    test_func_affine,
    x[now - 5 : now],
    ysmooth5[now - 5 : now],
    p0=[0, ysmooth5[now - 5]],
    )

    yresidual5 = ysmooth5[now - 20 : now] - test_func_affine(
    x[now - 20 : now], affineparams[0], affineparams[1]
    )

    residualparams, residualparams_covariance = optimize.curve_fit(
        test_func_wave,
        x[now - 15 : now],
        yresidual5.tail(15),
        p0=[0.1, 0.05, 1, 0],
        maxfev=100000,
        )

    def fitted_curve(x):
    function1 = test_func_affine(x + 3, affineparams[0], affineparams[1])
    function2 = test_func_wave(
        x + 3,
        residualparams[0],
        residualparams[1],
        residualparams[2],
        residualparams[3],
    )
        return function1 + function2

    xaffine = np.linspace(now - 15, now + 5, num=200, endpoint=True)
    xresidual = np.linspace(now - 15, now + 5, num=200, endpoint=True)
    yaffine = test_func_affine(xaffine + 2, affineparams[0], affineparams[1])
    yresidual = fitted_curve(xresidual)
    thisfigure = plt.figure()
    
    plt.plot(
        x[now - 21 : now + 10],
        y[now - 21 : now + 10],
        "b",
        xresidual,
        yresidual,
        "r",
        xaffine,
        yaffine,
        "g",
        x[now - 21 : now + 10],
        ysmooth5[now - 21 : now + 10],
        "y",
    )
    plt.axvline(now, linewidth=2, color="black", linestyle="--")
    plt.legend(["Points", "Fitted", "Affine"])
    plt.title(f"Sine Plot {timeframe} Date {myday}")
    plt.show()


# print(affineparams, residualparams)
# def fitted_sin(x):
#   myvalue = test_func(x, params[0], params[1], params[2], params[3])
#   return myvalue


# print(derivative(fitted_sin, 35, dx=1e-6))

IndentationError: expected an indented block (<ipython-input-385-914edc9b200b>, line 45)

In [ ]:
graph("2019-05-21", "AAPL", 25)

In [ ]:
## OLD for Reference
def buyorsell1(myday, mysymbol, timeframe):
    ts_price = getmydata(myday, mysymbol)

    if timeframe < 20:
        return 0

    try:
        now = timeframe
        x = np.linspace(0, now + 19, num=now + 20, endpoint=True)
        y = ts_price["close"]
        # ysmooth10 = ts_price["close10"]
        ysmooth5 = ts_price["close5"]

        def test_func_affine(x, m, k):
            return m * x + k

        def test_func_sin(x, a, b, c):
            return a * np.sin(b * x + c)

        def test_func_wave(x, a, b, c, d):
            return a * np.sin(b * x + c) + d

        # def test_func(x, a, b, c, d, m, k):
        #   return m * x + k + a * np.sin(b * x + c) + d

        affineparams, affineparams_covariance = optimize.curve_fit(
            test_func_affine,
            x[now - 15 : now],
            ysmooth5[now - 15 : now],
            p0=[1, 1],
        )

        yresidual5 = ysmooth5[now - 20 : now] - test_func_affine(
            x[now - 20 : now], affineparams[0], affineparams[1]
        )

        residualparams, residualparams_covariance = optimize.curve_fit(
            test_func_wave,
            x[now - 15 : now],
            yresidual5.tail(15),
            p0=[0.1, 0.05, 1, 0],
            maxfev=100000,
        )

        def fitted_curve(x):
            function1 = test_func_affine(x + 2, affineparams[0], affineparams[1])
            function2 = test_func_wave(
                x + 2,
                residualparams[0],
                residualparams[1],
                residualparams[2],
                residualparams[3],
            )
            return function1 + function2

        def first_derivative(x):
            return derivative(fitted_curve, x, dx=1e-3)

        def second_derivative(x):
            return derivative(first_derivative, x, dx=1e-3)

        # print (timeframe, first_derivative(timeframe), second_derivative(timeframe), 0.3*(affineparams[0] + residualparams[0]*residualparams[1]))
        # print (affineparams[0], residualparams[0], residualparams[1])

        # If wavelength > 60 periods, then don't trade
        if abs(residualparams[1]) < 0.1:
            print(f"Big Wavelength at period {timeframe}")
            return 0

        # If amplitude is too small, then don't trade
        if abs(residualparams[0]) < 0.001 * y[0]:
            print(f"Small Amplitude at period {timeframe}")
            return 0

        if abs(first_derivative(timeframe)) < (
            0.5 * (abs(affineparams[0]) + abs(residualparams[0] * residualparams[1]))
        ):
            if (second_derivative(timeframe)) > 0:
                return 1
            elif (second_derivative(timeframe)) < 0:
                return -1
            else:
                return 0
        else:
            return 0

    except:
        return 0.1

In [379]:
# old
def buyorsell2(myday, mysmbol, timeframe):

    ts_price = getmydata(myday, mysymbol)

    if timeframe < 20:
        return 0

    try:
        now = timeframe
        x = np.linspace(0, now + 19, num=now + 20, endpoint=True)
        y = ts_price["close"]
        # ysmooth10 = ts_price["close10"]
        ysmooth5 = ts_price["close5"]

        def test_func_affine(x, m, k):
            return m * x + k

        def test_func_sin(x, a, b, c):
            return a * np.sin(b * x + c)

        def test_func_wave(x, a, b, c, d):
            return a * np.sin(b * x + c) + d

        # def test_func(x, a, b, c, d, m, k):
        #   return m * x + k + a * np.sin(b * x + c) + d

        affineparams, affineparams_covariance = optimize.curve_fit(
            test_func_affine,
            x[now - 15 : now],
            ysmooth5[now - 15 : now],
            p0=[1, 1],
        )

        yresidual5 = ysmooth5[now - 20 : now] - test_func_affine(
            x[now - 20 : now], affineparams[0], affineparams[1]
        )

        residualparams, residualparams_covariance = optimize.curve_fit(
            test_func_wave,
            x[now - 15 : now],
            yresidual5.tail(15),
            p0=[0.1, 0.05, 1, 0],
            maxfev=100000,
        )

        def fitted_curve(x):
            function1 = test_func_affine(x + 2, affineparams[0], affineparams[1])
            function2 = test_func_wave(
                x + 2,
                residualparams[0],
                residualparams[1],
                residualparams[2],
                residualparams[3],
            )
            return function1 + function2

        def first_derivative(x):
            return derivative(fitted_curve, x, dx=1e-3)

        def second_derivative(x):
            return derivative(first_derivative, x, dx=1e-3)

        # print (timeframe, first_derivative(timeframe), second_derivative(timeframe), 0.3*(affineparams[0] + residualparams[0]*residualparams[1]))
        # print (affineparams[0], residualparams[0], residualparams[1])

        # If wavelength > 45 periods, then don't trade
        if abs(residualparams[1]) < 0.15:
            print(f"Big Wavelength at period {timeframe}")
            return 0

        # If amplitude is too small, then don't trade
        if abs(residualparams[0]) < 0.000001 * y[0]:
            print(f"Small Amplitude at period {timeframe}")
            return 0

        fd = first_derivative(timeframe)
        sd = second_derivative(timeframe)

        if abs(fd) < (
            0.1 * (abs(affineparams[0]) + abs(residualparams[0] * residualparams[1]))
        ):
            if sd > 0:
                return 1
            elif sd < 0:
                return -1
            else:
                return 0
        else:
            if fd > 0:
                if sd > 0:
                    return 1
                else:
                    return -1
            else:
                if sd < 0:
                    return -1
                else:
                    return 1
    except:
        return 0.1

In [380]:
# current version
def buyorsell(myday, mysmbol, timeframe):

    ts_price = getmydata(myday, mysymbol)

    if timeframe < 20:
        return 0

    try:
        now = timeframe
        x = np.linspace(0, now + 19, num=now + 20, endpoint=True)
        y = ts_price["close"]
        # ysmooth10 = ts_price["close10"]
        ysmooth5 = ts_price["close5"]

        def test_func_affine(x, m, k):
            return m * x + k

        def test_func_sin(x, a, b, c):
            return a * np.sin(b * x + c)

        def test_func_wave(x, a, b, c, d):
            return a * np.sin(b * x + c) + d

        # def test_func(x, a, b, c, d, m, k):
        #   return m * x + k + a * np.sin(b * x + c) + d

        affineparams, affineparams_covariance = optimize.curve_fit(
            test_func_affine,
            x[now - 5 : now],
            ysmooth5[now - 5 : now],
            p0=[0, ysmooth5[now - 5]],
        )

        yresidual5 = ysmooth5[now - 20 : now] - test_func_affine(
            x[now - 20 : now], affineparams[0], affineparams[1]
        )

        residualparams, residualparams_covariance = optimize.curve_fit(
            test_func_wave,
            x[now - 15 : now],
            yresidual5.tail(15),
            p0=[0.1, 0.05, 1, 0],
            maxfev=100000,
        )

        def fitted_curve(x):
            function1 = test_func_affine(x + 3, affineparams[0], affineparams[1])
            function2 = test_func_wave(
                x + 3,
                residualparams[0],
                residualparams[1],
                residualparams[2],
                residualparams[3],
            )
            return function1 + function2

        def first_derivative(x):
            return derivative(fitted_curve, x, dx=1e-3)

        def second_derivative(x):
            return derivative(first_derivative, x, dx=1e-3)

        # print (timeframe, first_derivative(timeframe), second_derivative(timeframe), 0.3*(affineparams[0] + residualparams[0]*residualparams[1]))
        # print (affineparams[0], residualparams[0], residualparams[1])

        # If trend is strong, then go with the flow.
        if (affineparams[0]) > ysmooth5[now] * 0.001:
            print("Buying trend")
            return 1
        if (affineparams[0]) < ysmooth5[now] * -0.001:
            print("Selling trend")
            return -1

        # If wavelength > 45 periods, then don't trade
        if abs(residualparams[1]) < 0.15:
            print(f"Big Wavelength at period {timeframe}")
            return 0

        # If amplitude is too small, then don't trade
        if abs(residualparams[0]) < 0.001 * ysmooth5[now]:
            print(f"Small Amplitude at period {timeframe}")
            return 0

        # stop trading at the end of day if none of the above
        if timeframe > 60:
            return 0

        fd = first_derivative(timeframe)
        sd = second_derivative(timeframe)

        if sd > 0:
            return 1
        if sd < 0:
            return -1
        return 0

    except:
        return 0.1

In [381]:
myday = "2019-05-28"
mysymbol = "TSLA"
timeframe = 56

graph(myday, mysymbol, timeframe)
buyorsell(myday, mysymbol, timeframe)

NameError: name 'fitted_curve' is not defined

In [ ]:
def tradeonedayonesymbolwithgraph(myday, mysymbol):
    ts_price = getmydata(myday, mysymbol)
    for timeframe in [*range(len(ts_price))]:
        print(timeframe, buyorsell(myday, mysymbol, timeframe))
        if timeframe > 19:
            graph(myday, mysymbol, timeframe)
    return 0

In [ ]:
myday = "2019-05-22"
mysymbol = "TSLA"
tradeonedayonesymbolwithgraph(myday, mysymbol)

In [ ]:
# simulation


def tradeonedayonesymbol(myday, mysymbol):
    ts_price = getmydata(myday, mysymbol)
    prices = []
    cash = 0
    pos = 0
    first_price = ts_price["open"][1]
    for timeframe in [*range(len(ts_price))]:
        #        print(myday, mysymbol, timeframe)
        bs = buyorsell(myday, mysymbol, timeframe)
        price = ts_price["close"][timeframe]

        if bs == 1:
            cash = cash - price
            pos = pos + 1
        if bs == -1:
            cash = cash + price
            pos = pos - 1

        print(timeframe, pos, price, cash, cash + pos * price)

        # if timeframe > 19:
        #    if timeframe < 65:
        #       graph(myday, mysymbol, timeframe)
        # print(
        #   timeframe,
        #  pos,
        # price,
        # (cash),
        # (cash + pos * price),
        # )
    last_price = ts_price["close"].values[-1]
    cash = cash + pos * last_price
    prices.append(pos * last_price)
    pos = 0
    print("Daily Position", pos, last_price, cash)
    print(100 * cash / last_price)
    return round(100 * cash / first_price, 2)


def tradealldaysonesymbol(daylist, symbol):
    profit = 0
    print(symbol)
    for day in daylist:
        daily_profit = tradeonedayonesymbol(day, symbol)
        profit = profit + daily_profit
        # print(day, " Daily: ", int(daily_profit), " Total to Date: ", int(profit))
    return profit


def tradeonedayallsymbols(day, symbollist):
    profit = 0
    print(day)
    for symbol in symbollist:
        symbol_profit = tradeonedayonesymbol(day, symbol)
        profit = profit + symbol_profit
        print(symbol, " Symbol Profit: ", int(symbol_profit), " Total: ", int(profit))
    return profit


def simulate(daylist, symbollist):
    overallprofit = 0
    for symbol in symbollist:
        symbol_profit = tradealldaysonesymbol(daylist, symbol)
        overallprofit = overallprofit + symbol_profit
        print(
            symbol,
            " Symbol Profit: ",
            int(symbol_profit),
            " Overall Profit: ",
            int(overallprofit),
        )
    return overallprofit

In [357]:
myday = "2019-05-28"
mysymbol = "GOOGL"
tradeonedayonesymbol(myday, mysymbol)

0 0 1140.62 0 0.0
1 0 1144.79 0 0.0
2 0 1145.49 0 0.0
3 0 1146.48 0 0.0
4 0 1145.285 0 0.0
5 0 1147.0 0 0.0
6 0 1145.21 0 0.0
7 0 1145.77 0 0.0
8 0 1146.86 0 0.0
9 0 1144.87 0 0.0
10 0 1145.7 0 0.0
11 0 1147.2 0 0.0
12 0 1147.3885 0 0.0
13 0 1148.14 0 0.0
14 0 1150.65 0 0.0
15 0 1149.18 0 0.0
16 0 1148.64 0 0.0
17 0 1150.3 0 0.0
18 0 1152.44 0 0.0
19 0 1152.91 0 0.0
Small Amplitude at period 20
20 0 1153.957 0 0.0
Small Amplitude at period 21
21 0 1153.535 0 0.0
Small Amplitude at period 22
22 0 1152.68 0 0.0
Big Wavelength at period 23
23 0 1149.89 0 0.0
Small Amplitude at period 24
24 0 1150.04 0 0.0
Small Amplitude at period 25
25 0 1151.38 0 0.0
Small Amplitude at period 26
26 0 1152.54 0 0.0
Small Amplitude at period 27
27 0 1153.73 0 0.0
28 1 1152.465 -1152.465 0.0
29 0 1154.04 1.5750000000000455 1.5750000000000455
30 -1 1155.15 1156.7250000000001 1.5750000000000455
31 -2 1155.13 2311.8550000000005 1.5950000000002547
32 -3 1154.84 3466.6950000000006 2.1750000000010914
Small Ampli

-0.63

In [ ]:
tradealldaysonesymbol(trading_days, ("AAPL"))

In [ ]:
# Flip flop between long and short positions


def tradeonedayonesymbol(myday, mysymbol):
    ts_price = getmydata(myday, mysymbol)
    prices = []
    cash = 0
    pos = 0
    first_price = ts_price["open"][0]
    # for timeframe in [*range(len(ts_price))]:
    for timeframe in [*range(len(ts_price))]:
        print(myday, mysymbol, timeframe)
        price = ts_price["open"][timeframe]
        if shouldbuy(timeframe):
            if pos == 0:
                cash = cash - price
                pos = pos + 1
            if pos < 0:
                cash = cash - 2 * price
                pos = pos + 2
        if shouldsell(timeframe):
            if pos == 0:
                cash = cash + price
                pos = pos - 1
            if pos > 0:
                cash = cash + 2 * price
                pos = pos - 2
        # print(
        #   timeframe,
        #  pos,
        # should_be_long(timeframe),
        # int(price),
        # int(cash),
        # int(cash + pos * price),
        # )
    last_price = ts_price["close"].values[-1]
    cash = cash + pos * last_price
    prices.append(pos * last_price)
    pos = 0
    # print(pos, last_price, cash)
    # print(100 * cash / last_price)
    return round(100 * cash / first_price, 2)


def tradealldaysonesymbol(daylist, symbol):
    profit = 0
    print(symbol)
    for day in daylist:
        daily_profit = tradeonedayonesymbol(day, symbol)
        profit = profit + daily_profit
        # print(day, " Daily: ", int(daily_profit), " Total to Date: ", int(profit))
    return profit


def tradeonedayallsymbols(day, symbollist):
    profit = 0
    print(day)
    for symbol in symbollist:
        symbol_profit = tradeonedayonesymbol(day, symbol)
        profit = profit + symbol_profit
        print(symbol, " Symbol Profit: ", int(symbol_profit), " Total: ", int(profit))
    return profit


def simulate(daylist, symbollist):
    overallprofit = 0
    for symbol in symbollist:
        symbol_profit = tradealldaysonesymbol(daylist, symbol)
        overallprofit = overallprofit + symbol_profit
        print(
            symbol,
            " Symbol Profit: ",
            int(symbol_profit),
            " Overall Profit: ",
            int(overallprofit),
        )
    return overallprofit